# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat publications.csv















,,,,,,,

In [10]:
!cat publications.

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2009-10-01	Paper Title Number 1	Journal 1	This paper is about the number 1. The number 2 is left for future work.	Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).	paper-title-number-1	http://academicpages.github.io/files/paper1.pdf	http://academicpages.github.io/files/slides1.pdf
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf	http://academicpages.github.io/files/slides2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf	http://academicpages.github.io/files/slides3.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [5]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
publications = pd.read_csv("publications.csv", header=0)
publications


,pub_date,title,venue,exerpt,citation,url_slug,paper_url,slides_url
0,2024-06-03,Inferring interaction potentials from stochast...,In review at Physical Review Research,NaN,"Ella M. King, Megan Engel, Caroline Martin, Sa...",inferring-potentials,NaN,http://carolinesmartin.github.io/files/inferri...
1,2023-09-20,Precise characterization of nanometer-scale sy...,Applied Optics,NaN,"Xander M. de Wit, Amelia W. Paine, Caroline Ma...",iscat,NaN,http://carolinesmartin.github.io/files/iscat.pdf
2,2023-05-15,WOW STEM: Addressing the STEM gender gap with ...,"American Philosophical Society, Women in Scien...",NaN,"Caroline Martin, Madelyn Leembruggen, and Tayl...",wow-stem,NaN,http://carolinesmartin.github.io/files/wow-ste...
3,2022-10-22,In-line holographic microscopy with model-base...,Nature Reviews Methods Primers,NaN,"Caroline Martin, Lauren E. Altman, Siddharth R...",in-line-holographic-microscopy,NaN,http://carolinesmartin.github.io/files/in-line...
4,2022-04-01,What’s for Lunch? A systematic ordering of foo...,arXiv.org,NaN,Madelyn Leembruggen and Caroline Martin. 4/1/2...,whats-for-lunch,NaN,http://carolinesmartin.github.io/files/whats-f...
5,2021-06-07,Improving holographic particle characterizatio...,Optics Express,NaN,"Caroline Martin, Brian Leahy, and Vinothan N. ...",spherical-aberration,NaN,http://carolinesmartin.github.io/files/spheric...
6,2020-01-20,Large depth-of-field tracking of colloidal sph...,Optics Express,NaN,"Brian Leahy, Ronald Alexander, Caroline Martin...",modelling-objective,NaN,http://carolinesmartin.github.io/files/modelli...
7,2018-04-20,Application of Bayesian Inference to Analysis ...,Wellesley College,NaN,Caroline Martin. 6/5/2018. “Application of Bay...,bayesian-dls,NaN,http://carolinesmartin.github.io/files/bayesia...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [15]:
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    print(md_filename)

2024-06-03-inferring-potentials.md
2023-09-20-iscat.md
2023-05-15-wow-stem.md
2022-10-22-in-line-holographic-microscopy.md
2022-04-01-whats-for-lunch.md
2021-06-07-spherical-aberration.md
2020-01-20-modelling-objective.md
2018-04-20-bayesian-dls.md


## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [16]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    #if len(str(item.excerpt)) > 5:
    #    md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    #if len(str(item.excerpt)) > 5:
    #    md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download paper here](" + item.slides_url + ")\n" 

    #if len(str(item.paper_url)) > 5:
    #    md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [17]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md
2010-10-01-paper-title-number-2.md
2015-10-01-paper-title-number-3.md
2018-04-20-bayesian-dls.md
2020-01-20-modelling-objective.md
2021-06-07-spherical-aberration.md
2022-04-01-whats-for-lunch.md
2022-10-22-in-line-holographic-microscopy.md
2023-05-15-wow-stem.md
2023-09-20-iscat.md
2024-02-17-paper-title-number-4.md
2024-06-03-inferring-potentials.md


In [18]:
!cat ../_publications/2021-06-07-spherical-aberration.md

---
title: "Improving holographic particle characterization by modeling spherical aberration"
collection: publications
permalink: /publication/2021-06-07-spherical-aberration
date: 2021-06-07
venue: 'Optics Express'
slidesurl: 'http://carolinesmartin.github.io/files/spherical-aberration.pdf'
citation: 'Caroline Martin, Brian Leahy, and Vinothan N. Manoharan. 2021. “Improving holographic particle characterization by modeling spherical aberration.” Opt. Express, 29, 12, Pp. 18212–18223.'
---
[Download paper here](http://carolinesmartin.github.io/files/spherical-aberration.pdf)

Recommended citation: Caroline Martin, Brian Leahy, and Vinothan N. Manoharan. 2021. “Improving holographic particle characterization by modeling spherical aberration.” Opt. Express, 29, 12, Pp. 18212–18223.